In [ ]:
#| default_exp evaluation

# Datasets Evaluation

In [ ]:
#| export
import pickle
from functools import partial
from inspect import signature
from typing import Any, Callable, Dict, List, Optional, Union

import numpy as np
import pandas as pd
from fugue import transform
from fugue.collections.yielded import Yielded
from fugue.constants import FUGUE_CONF_WORKFLOW_EXCEPTION_INJECT
from fugue.dataframe import DataFrame
from fugue.workflow import FugueWorkflow

In [ ]:
#| export
def _cotransform(
    df1: Any,
    df2: Any,
    using: Any,
    schema: Any = None,
    params: Any = None,
    partition: Any = None,
    engine: Any = None,
    engine_conf: Any = None,
    force_output_fugue_dataframe: bool = False,
    as_local: bool = False,
) -> Any:
    dag = FugueWorkflow(compile_conf={FUGUE_CONF_WORKFLOW_EXCEPTION_INJECT: 0})
    src = dag.create_data(df1).zip(dag.create_data(df2), partition=partition)
    tdf = src.transform(
        using=using,
        schema=schema,
        params=params,
        pre_partition=partition,
    )
    tdf.yield_dataframe_as("result", as_local=as_local)
    dag.run(engine, conf=engine_conf)
    result = dag.yields["result"].result  # type:ignore
    if force_output_fugue_dataframe or isinstance(df1, (DataFrame, Yielded)):
        return result
    return result.as_pandas() if result.is_local else result.native

In [ ]:
#| export
def _quantiles_from_levels(model_name, level):
    """Returns quantiles associated to `level` and the sorte columns of `model_name`"""
    level = sorted(level)
    alphas = [100 - lv for lv in level]
    quantiles = [alpha / 200 for alpha in reversed(alphas)]
    quantiles.extend([1 - alpha / 200 for alpha in alphas])
    cols = [f'{model_name}-lo-{lv}' for lv in reversed(level)]
    cols.extend([f'{model_name}-hi-{lv}' for lv in level])
    return np.array(quantiles), cols

In [ ]:
#| hide
from fastcore.test import test_eq, test_fail, test_close, test_warns

In [ ]:
#| hide
levels = range(101)
quantiles, cols = _quantiles_from_levels('mymodel', list(levels))
test_close(
    np.linspace(0., 1., num=len(quantiles)),
    quantiles, eps=1e-2
)
test_eq(
    cols,
    [f'mymodel-lo-{lv}' for lv in reversed(levels)] + [f'mymodel-hi-{lv}' for lv in levels]
)

In [ ]:
#| export
def _evaluate(
        df: pd.DataFrame, 
        df_train: pd.DataFrame,
        metrics: Optional[List[Callable]] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
    ) -> pd.DataFrame:
    cols_to_rm = '|'.join([id_col, time_col, target_col, 'cutoff', 'lo', 'hi'])
    has_cutoff = 'cutoff' in df.columns
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    y = df[target_col].values
    eval_ = {}
    for model in models:
        eval_[model] = {}
        for metric in metrics:
            y_hat = df[model].values
            metric_name = metric.__name__
            params = signature(metric).parameters
            if 'y_train' in params:
                if df_train is None:
                    raise Exception(f'Please provide `Y_df` to compute {metric_name}')
                eval_[model][metric_name] = metric(y, y_hat, y_train=df_train[target_col].values)
            elif 'quantiles' in params:
                if level is None:
                    raise Exception(
                        f'Please provide the `level` argument to compute {metric_name}. '
                    )
                quantiles, lv_cols = _quantiles_from_levels(model_name=model, level=level)
                y_hat_q = df[lv_cols].values
                eval_[model][metric_name] = metric(y, y_hat_q, quantiles=quantiles)
            elif ('y_hat_lo' in params) and ('y_hat_hi' in params):
                if level is None:
                    raise Exception(
                        f'Please provide the `level` argument to compute {metric_name}. '
                    )
                for lv in level:
                    y_hat_lo = df[f'{model}-lo-{lv}'].values
                    y_hat_hi = df[f'{model}-hi-{lv}'].values
                    eval_[model][f'{metric_name}-lv-{lv}'] = metric(y, y_hat_lo, y_hat_hi)
            else:
                eval_[model][metric_name] = metric(y, y_hat)
    eval_df = pd.DataFrame(eval_).rename_axis('metric').reset_index()
    if has_cutoff:
        eval_df.insert(0, 'cutoff', df['cutoff'].iloc[0])
    eval_df.insert(0, id_col, df[id_col].iloc[0])
    return eval_df

In [ ]:
#| export
def _evaluate_without_insample(
        df: pd.DataFrame, 
        metrics: Optional[List[Callable]] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
    ) -> pd.DataFrame:
    return _evaluate(
        df=df, df_train=None, metrics=metrics, id_col=id_col, time_col=time_col,
        target_col=target_col, level=level,
    )

In [ ]:
#| export
def _schema_evaluate(
        df: pd.DataFrame,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
    ) -> str: 
    cols_to_rm = '|'.join([id_col, time_col, target_col, 'cutoff', 'lo', 'hi'])
    has_cutoff = 'cutoff' in df.columns
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    str_models = ','.join([f"{model}:double" for model in models])
    dtypes = df.dtypes
    id_col_type = dtypes.loc[id_col]
    if id_col_type == 'category':
        raise NotImplementedError(
            'Use of `category` type to identify each time series is not yet implemented. '
            f'Please transform your {id_col} to string to continue.'
        )
    id_col_type = 'string' if id_col_type == 'object' else id_col_type
    cutoff_col_type = ''
    if has_cutoff:
        cutoff_col_type = f"{dtypes.loc['cutoff']}".replace('64[ns]', '')
        cutoff_col_type = 'string' if cutoff_col_type == 'object' else cutoff_col_type
    schema = (
        f'{id_col}:{id_col_type},metric:string,'
        + (f'cutoff:{cutoff_col_type},' if has_cutoff else '')
        + str_models
    )
    return schema

In [ ]:
#| export
def _agg_evaluation(
        df_eval: pd.DataFrame, 
        agg_fn: Any, 
        agg_by: Any,
    ) -> pd.DataFrame:
    cols_to_rm = '|'.join(agg_by + ['unique_id', 'metric', 'cutoff', 'lo', 'hi'])
    models = df_eval.loc[:, ~df_eval.columns.str.contains(cols_to_rm)].columns
    return df_eval.groupby(agg_by)[models].apply(agg_fn, axis=0).reset_index()

In [ ]:
#| export
def _schema_agg_evaluation(
        df: pd.DataFrame, 
        agg_by: Optional[List[str]] = None,
    ) -> str:
    cols_to_rm = '|'.join(agg_by + ['unique_id', 'metric', 'cutoff', 'lo', 'hi'])
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    str_models = ','.join([f'{model}:double' for model in models])
    dtypes = df.dtypes
    agg_by_types = [dtypes.loc[col] for col in agg_by]
    if 'category' in agg_by_types:
        raise NotImplementedError(
            'Use of `category` type is not yet implemented. '
            f'Please transform your columns to string to continue.'
        )
    agg_by_types = ['string' if col == 'object' else col for col in agg_by_types]
    agg_by_types = [col.replace('64[ns]', '') for col in agg_by_types]
    schema = [f'{col}:{type_}' for col, type_ in zip(agg_by, agg_by_types)]
    schema = ','.join(schema) + ',' + str_models
    return schema

In [ ]:
#| hide
from datasetsforecast.m4 import M4, M4Evaluation

In [ ]:
#| hide
fforma_url = 'https://github.com/Nixtla/m4-forecasts/raw/master/forecasts/submission-245.zip'
M4Evaluation.load_benchmark('data', 'Hourly', fforma_url)
eval_test = pd.read_csv('data/m4/datasets/submission-245.csv').query("id.str.startswith('H')")
eval_test = eval_test.set_index('id')
eval_test.columns = list(range(1, 49))
eval_test = eval_test.unstack().reset_index()
eval_test.columns = ['ds', 'unique_id', 'FFORMA']
eval_test = eval_test[['unique_id', 'ds', 'FFORMA']].sort_values(['unique_id', 'ds'])
hourly, *_ = M4.load('data', 'Hourly')
test_hourly = hourly.groupby('unique_id').tail(48)
train_hourly = hourly.drop(test_hourly.index)
test_hourly['FFORMA'] = eval_test['FFORMA'].values

/var/folders/7x/1l3vkh3x4_q3s4r36b60s8f00000gn/T/ipykernel_13221/3530693183.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_hourly['FFORMA'] = eval_test['FFORMA'].values


In [ ]:
#| hide
test_eq(
    _schema_evaluate(eval_test),
    'unique_id:string,metric:string,FFORMA:double'
)

In [ ]:
#| export
def accuracy(
        Y_hat_df: pd.DataFrame,
        metrics: List[Callable],
        Y_test_df: Optional[pd.DataFrame] = None,
        Y_df: Optional[pd.DataFrame] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
        agg_by: Optional[List[str]] = None,
        agg_fn: Callable = np.mean,
        engine: Any = None,
        **transform_kwargs: Any,
    ) -> pd.DataFrame:
    """Evaluate forecast using different metrics.
    
    Parameters
    ----------
    Y_hat_df : pandas DataFrame
        Forecasts and models to evaluate.
        Can contain the actual values given by `target_col`.
    metrics : List of Callables
        Functions with arguments `y`, `y_hat`, and optionally `y_train`.
    Y_test_df :  pandas DataFrame, optional (default=None)
        True values. 
        Nedded if `Y_hat_df` does not have the true values.
    Y_df : pandas DataFrame, optional (default=None)
        Training set. Used to evaluate metrics such as `mase`. 
    id_col : str (default='unique_id')
        Column that identifies each serie. If 'index' then the index is used.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.
    target_col : str (default='y')
        Column that contains the target.
    agg_by: List[str], optional (default=None)
        List of columns to aggregate the results.
        To get metrics per time series use [`id_col`].
    agg_fn: Callable, (default=np.mean)
        Function to aggregate metrics.
    engine: Any
        Engine to distributed computing.
    transform_kwargs: Any
        Extra arguments to pass to fugue's `transform`.
        
    Returns
    -------
    result : pandas DataFrame
        Metrics with one column per model.
    """
    if 'y' not in Y_hat_df.columns:
        raise Exception(
            'Please include the actual values in `Y_hat_df` '
            'or pass `Y_test_df`.'
        )
    df = Y_hat_df if Y_test_df is None else Y_hat_df.merge(Y_test_df, how='left', on=[id_col, time_col])
    transform_fn = partial(_cotransform, df2=Y_df) if Y_df is not None else transform   
    if Y_df is None:
        fn = _evaluate_without_insample
    else:
        fn = _evaluate
    has_cutoff = 'cutoff' in Y_hat_df.columns
    evaluation_df = transform_fn(
        df, 
        using=fn, 
        engine=engine, 
        params=dict(
            metrics=metrics,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            level=level,
        ), 
        schema=_schema_evaluate(
            df, 
            id_col=id_col, 
            time_col=time_col, 
            target_col=target_col,
        ), 
        partition=dict(by=id_col) if not has_cutoff else dict(by=[id_col, 'cutoff']),
    )
    if agg_by is not None:
        agg_by = ['metric'] + agg_by
    else:
        agg_by = ['metric']
    evaluation_df = transform(
        evaluation_df,
        using=_agg_evaluation,
        engine=engine,
        params=dict(agg_fn=agg_fn, agg_by=agg_by),
        schema=_schema_agg_evaluation(evaluation_df, agg_by),
        partition=agg_by,
        **transform_kwargs,
    )
    return evaluation_df

In [ ]:
#| hide
from datasetsforecast.losses import smape, mase, mape, mqloss, mae, scaled_crps, coverage

In [ ]:
#| hide
def mase_24(y, y_hat, y_train):
    return mase(y, y_hat, y_train, seasonality=24)

In [ ]:
#| hide
test_cutoff = pd.concat([test_hourly.assign(cutoff=str(i), model1=0) for i in range(3)])
eval_cutoff = accuracy(test_cutoff, metrics=[smape, mape], agg_by=['cutoff'])
test_eq(
    eval_cutoff.shape,
    (6, 4)
)

In [ ]:
#| hide
test_close(
    accuracy(test_hourly, Y_df=train_hourly, metrics=[smape, mase_24])['FFORMA'].values,
    np.array([0.818598, 11.505702])
)

In [ ]:
#| hide
test_eq(
    accuracy(test_hourly, Y_df=train_hourly, metrics=[smape]),
    accuracy(test_hourly, metrics=[smape])
)

In [ ]:
#| hide
test_fail(
    lambda : accuracy(test_hourly, metrics=[smape, mase]), 
    contains='mase'
)

ERROR:root:_1 _State.RUNNING -> _State.FAILED  Please provide `Y_df` to compute mase


In [ ]:
#| hide
test_fail(
    lambda : accuracy(test_hourly, metrics=[smape, mqloss]), 
    contains='mqloss'
)

ERROR:root:_1 _State.RUNNING -> _State.FAILED  Please provide the `level` argument to compute mqloss. 


In [ ]:
#| hide
# add level forecasting
test_hourly_series = test_hourly.query('unique_id == "H1"')
level = [50, 80, 90, 99]
for lv in level:
    alpha = (100 - lv) / 200
    test_hourly_series[f'FFORMA-lo-{lv}'] = test_hourly_series['y'] - alpha
    test_hourly_series[f'FFORMA-hi-{lv}'] = test_hourly_series['y'] + alpha
evaluation = accuracy(
    test_hourly_series, 
    metrics=[smape, mqloss, mae, scaled_crps, coverage], level=level
)
test_close(
    evaluation.query('metric == "mqloss"')['FFORMA'].iloc[0],
    np.mean([((100 - lv)/ 200) ** 2 for lv in level])
)
test_close(
    evaluation.query('metric == "scaled_crps"')['FFORMA'].iloc[0],
    np.mean([2 * (((100 - lv)/ 200) ** 2) * len(test_hourly_series)/test_hourly_series['y'].sum() for lv in level])
)

/var/folders/7x/1l3vkh3x4_q3s4r36b60s8f00000gn/T/ipykernel_13221/664392555.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_hourly_series[f'FFORMA-lo-{lv}'] = test_hourly_series['y'] - alpha
/var/folders/7x/1l3vkh3x4_q3s4r36b60s8f00000gn/T/ipykernel_13221/664392555.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_hourly_series[f'FFORMA-hi-{lv}'] = test_hourly_series['y'] + alpha
/var/folders/7x/1l3vkh3x4_q3s4r36b60s8f00000gn/T/ipykernel_13221/664392555.py:7: SettingWithCopyWarning: 
A value 

In [ ]:
#| hide
# test coverage
y = np.random.normal(size=1_000)
test_df = pd.DataFrame({
    'unique_id': '1', 
    'ds': np.arange(1_000).astype(int), 
    'model1': y,
    'y': y,
})
level = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for lv in level:
    alpha_lo = (100 - lv) / 200
    alpha_hi = 1 - alpha_lo
    test_df[f'model1-lo-{lv}'] = np.quantile(test_df['y'], q=alpha_lo)
    test_df[f'model1-hi-{lv}'] = np.quantile(test_df['y'], q=alpha_hi)
evaluation = accuracy(
    test_df, 
    metrics=[mqloss, scaled_crps, coverage], level=level
)

In [ ]:
evaluation

,metric,model1
0,coverage-lv-0,0.000000
1,coverage-lv-10,0.100000
2,coverage-lv-100,1.000000
3,coverage-lv-20,0.200000
4,coverage-lv-30,0.300000
5,coverage-lv-40,0.400000
6,coverage-lv-50,0.500000
7,coverage-lv-60,0.600000
8,coverage-lv-70,0.700000
9,coverage-lv-80,0.800000
